[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/enigma-brain/polars_workshop_2025/blob/main/polars.ipynb)

# Polars Intro Workshop

## Learning objectives
- why is Enigma using Polars now? 
- understand advantages of Polars, basics of syntax, what's possible

## Why Polars

We're switching data stacks: MySQL + DataJoint -> Delta Lake tables + Polars. Several tools can analyze Delta Lake tables, but Polars offers a good balance of [performance](https://pola.rs/posts/benchmarks/) and simplicity.

Orcapod uses Polars.

Generally useful if you're working with 2D tables.

Gaining [popularity](https://www.linkedin.com/posts/jeroenjanssens_polars-just-passed-30k-stars-on-github-congratulations-activity-7256626437082746880-fguS/).

## Special features of Polars

#### Performance and memory
- Columnar architecture
    - stores and processes data by column rather than by row
- Parallelism by default 
- Lazy API
    - queries aren't immediately executed; Polars optimizes the entire plan first
- Streaming
    - process datasets larger than available RAM by working in chunks

## Core features for data manipulation

Understanding these is most of the work. Extending to Lazy API and streaming is trivial.

#### Data I/O
- python objects: dict of lists, Pandas DataFrame
- CSV, Excel, Parquet
- Databases
- Delta Lake tables
- Cloud storage
- others ([Docs](https://docs.pola.rs/user-guide/io/), [API reference](https://docs.pola.rs/api/python/dev/reference/io.html))

#### Single table
- `filter` and `select`: filter rows and select columns
- `with_columns`: create new columns based on values in other columns
- `group_by` and `agg`: group data and compute stats
- `pivot` and `unpivot`: convert between long and wide format

#### Multi table
- `join` and `concat`: join and concatenation

## Limitations
- newer: smaller community, fewer tutorials
- it's compared to Pandas, but the syntax is pretty different (although quite clear)


In [ ]:
# !pip install palmerpenguins
import palmerpenguins as pp
import polars as pl
pl.Config.set_fmt_str_lengths(100) # nice for displaying longer strings and not truncate them

df = pl.from_pandas(pp.load_penguins())


## Get basic understanding of data
- `head`, `tail`
- `glimpse`
- `sample`
- `schema`
- `describe`

# Data manipulation/querying

Assuming we have a dataframe called `df`, using Polars generally involves the format:
```
df.context(expression)
```
where the 'context' can be one of the following:

- `select` columns
- `with_columns` to add columns to data frame
- `filter` rows
- `group_by` and `agg` to group rows by their values, compute statistics

## `select` columns

In [ ]:
# grab a column
df.select("body_mass_g")

# use pl.col to explicitly create an expression that represents a column
df.select(pl.col("body_mass_g"))

# can extend pl.col to sort and do arithmetic, 
df.select(pl.col("body_mass_g").sort() / 1000)

# but further operations require more parentheses, e.g. renaming the column
df.select(
    (pl.col("body_mass_g")
     .sort() 
     / 1000)
     .alias("body_mass_kg")
) 

# rename using a 'named expression'
df.select(
    body_mass_kg = (pl.col("body_mass_g")
     .sort() 
     / 1000)
) 

# alias method nice for storing expressions in variables  
kg_expr = ((pl.col("body_mass_g").sort() / 1000).alias("body_mass_kg"))
df.select(kg_expr)

# select multiple columns
df.select(
    (pl.col("species"), pl.col("island")) # or ["species", "island"]
).unique().sort(by="species")


There are tons of methods to use within expressions (like `.sort`) that can be used in expressions. See [here](https://docs.pola.rs/api/python/stable/reference/expressions/index.html) for a full list.

Problem: using `select`, create a column that computes normalized deviations from the mean body mass, and name this column "body_mass_zscore". 

In [ ]:
# expressions support broadcasting
kg_expr = ((pl.col("body_mass_g") - pl.col("body_mass_g").mean()) / pl.col("body_mass_g").std()).alias("body_mass_zscore")
df.select(kg_expr)

## `with_columns` to add columns

In [ ]:
# create two columns to add!
kg_expr = (pl.col("body_mass_g") / 1000).alias("body_mass_kg")
swim_score_expr = (pl.col("flipper_length_mm") / pl.col("body_mass_kg")).alias("swim_score")

# Polars executes expressions in the same context in parallel 
df.with_columns(
    kg_expr,
    swim_score_expr
)

# Columns must be creted in serial, by using `with_columns` context twice
df = df.with_columns(
    kg_expr,
).with_columns(
    swim_score_expr,
)

# note we overwrote the original dataframe
df

## `filter` rows

In [ ]:
df.filter(pl.col("species") == "Gentoo")

# filter based on multiple conditions
df.filter((pl.col("species") == "Adelie") & (pl.col("island") == "Biscoe"))

## can use commas instead of ampersands, but not 'and'!
df.filter(
    (pl.col("species") == "Adelie"), 
    (pl.col("island") == "Biscoe"),
)

# in addition to storing expressions as variables to keep things organized, you can also define functions to create expressions
def filter_species_island_expr(species, island):
    return (
        pl.col("species") == species,
        pl.col("island") == island
    )

df.filter(
    filter_species_island_expr("Adelie", "Biscoe")
    )

Contexts can be chained together using:
```
df.context1(expression1).context2(expression2)
```

Problem 1: Find the Chinstrap penguin with the largest swim_score. Return just the swim_score

Problem 2: Find the sex of the Chinstrap penguin with the largest swim score. Return just the sex. 

In [ ]:
df.filter(pl.col("species") == "Chinstrap").select(pl.col("swim_score").max().alias("max_swim_score"))

In [ ]:
(
df.select(pl.col(["species", "sex", "swim_score"]))
    .filter(pl.col("species") == "Chinstrap")
    .sort(by="swim_score", descending=True)
    .head(1)
    .select(pl.col("sex"))
)

## `groupby` and `agg` to group rows by their values, compute statistics

In [ ]:
df.group_by("species").agg(
    pl.col("body_mass_kg").mean(),
    pl.col("swim_score").mean(),
).sort(by="species")

# group by multiple columns, rename the columns
df.group_by(["species", "sex"]).agg(
    pl.col("body_mass_kg").mean().alias("mean_body_mass_g"),
    pl.col("swim_score").mean().alias("mean_swim_score"),
).sort(by="species")

# drop nulls
(
    df
    .drop_nulls(subset = ["sex"])
    .group_by(["species", "sex"])
    .agg(
        pl.col("body_mass_kg").mean().alias("mean_body_mass_kg"),
        pl.col("swim_score").mean().alias("mean_swim_score"),
        pl.len().alias("count") # count the number of rows in each group
    )
    .sort(by="species")
)

# 'expression expansions' help remove redundant code
(
    df
    .drop_nulls(subset = ["sex"])
    .group_by(["species", "sex"])
    .agg(
        pl.col("body_mass_kg", "swim_score").mean().name.prefix("mean_"), # can also use .name.suffix("_mean")
        pl.len().alias("count")
    )
    .sort(by="species")
)

Problem: use the .quantile method to compute the 0.25, 0.5, and 0.75 quantile per species. Name these columns "q1_swim_score", "q2_swim_score", etc.
Bonus if you can avoid having .quantile in your code

In [ ]:
df.group_by(["species"]).agg(
        pl.col("swim_score").quantile(0.25).alias("q1_swim_score"),
        pl.col("swim_score").quantile(0.5).alias("q2_swim_score"),
        pl.col("swim_score").quantile(0.75).alias("q3_swim_score"),
    )

# more clever but less explicit
df.group_by("species").agg([
    pl.col("swim_score").quantile(q).alias(f"q{i}_swim_score") 
    for i, q in enumerate([0.25, 0.5, 0.75], 1)
])

# Pivots: wide format <-> long format

"Wide" tables have a column for each variable. "Long" tables have fewer columns, with a single column for variable type and another for the corresponding value.

- `.pivot()` : long -> wide
- `.unpivot()` : wide -> long



Our dataframe is already in wide format. Let's convert it to long format.

In [ ]:
# go from wide to long format
df_long = (
            df.select("species", "island", "bill_length_mm", "bill_depth_mm")
            .unpivot(on=["bill_length_mm", "bill_depth_mm"], # columns to collapse into "variable" column
                    index=["species", "island"]) # used as identifier variables
)

df_long.sample(5)

Once in long format, plotting libraries like seaborn make it easy to visualize statistics with respect to these variables!

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="darkgrid")
plt.figure(figsize=(12, 6))
sns.violinplot(data=df_long, x="species", y="value", hue="variable")
plt.title("Distribution of Bill Measurements by Species")
plt.xlabel("Species")
plt.ylabel("Measurement Value")
plt.legend(title="Measurement Type")
plt.show()
sns.despine()

Going from wide to long format involves creating one-to-many relationships: the column name is now represented many times in the variable column.

Alternatively, going from long to wide format involves creating many-to-one relationships. As a result, you need to also specify an 'aggregate_function' to tell Polars what to do with duplicates.

In [ ]:
df.pivot(on="species", # will become the column values
         index="sex", # will become the row values
         values="body_mass_kg", # will become the values in the pivot table 
         aggregate_function="max")

Problem: confirm with command we learned above that the max body_mass_kg for a male Adelie is that value

In [ ]:
# exercise, confirm this for Gentoo
df.filter(
    (pl.col("species") == "Adelie")
    ).select(pl.col("body_mass_kg")
    .max()
    )
# or
df.select(
    pl.col("body_mass_kg")
    .filter(pl.col("species") == "Adelie")
    .max()

    
)

In [ ]:
# convert back to wide format, but this fails becaue you don't know which bill length+depth go together for each individual
df_long.pivot(on="variable",
            index=["species", "island"],
            values="value",
            aggregate_function="first") # looking at the options available, this seems most appropriate?

In [ ]:
# Redo, preserving penguin ID
# go from wide to long format
df_long2 = (df
            .select("species", "island", "bill_length_mm", "bill_depth_mm")
            .with_row_index("penguin_id")
            .unpivot(["bill_length_mm", "bill_depth_mm"],
                index=["species", "island", "penguin_id"])
)
df_long2

In [ ]:
# Now we can successfully go back to wide format
df_long2.pivot(on="variable",
            index=["species", "island", "penguin_id"],
            values="value",
            aggregate_function="first")

# Lazy frames
For exploratory data analyses, use DataFrames. For performance, use LazyFrames.

Syntax same as before, but results not immediately available. Need to use `.collect()`.

To load in a LazyFrame, either convert an existing DataFrame or use one of the `.scan_*()` methods

In [ ]:
df_lazy = pl.LazyFrame(df)
# df_lazy = df.lazy()
df_lazy
# note: methods like df_lazy.head() now behave differenlty

In [ ]:
kg_expr = (pl.col("body_mass_g") / 1000).alias("body_mass_kg")
gentoo_male_expr = (pl.col("species") == "Gentoo") , (pl.col("sex") == "male")

# make lazy query that gets the body mass in kg for Gentoo males
lazy_query = (
    df_lazy.with_columns(kg_expr)
    .filter(gentoo_male_expr)
    .select("body_mass_kg")
)

# returns another lazy frame
lazy_query

This lazy query, which hasn't been optimized, is read from the bottom up. 

$\pi$ stands for PROJECTION and refers to columns. We start with all 10 columns and end up with 1. 

Contexts are listed in the same order as written in our query.

Let's compare this to the optimized query plan below.

In [ ]:
lazy_query.show_graph()

Polars uses predicate and projection pushdown, database optimization techniques, to make the selection of columns (projection) and the filtering of rows (predicate) as early as possible so that it only loads the data you need.

You can also use `.explain()` to get a string representation of the query plan (useful if text gets truncated within graph nodes).

In [ ]:
lazy_query.explain()

In [ ]:
# returns a regular polars data frame, not a lazy frame
query_results = lazy_query.collect()
print(type(lazy_query))
print(type(query_results))

In [ ]:
query_results.describe()

# Streaming

Default is to analyze all the data in memory. For streaming the data in batches, just specify as an argument in .collect()

In [ ]:
lazy_query.collect(engine="streaming")

# Misc Advanced Stuff
## Loading multiple MySQL tables into Polars

In [ ]:
# !pip install connectorx
from urllib.parse import quote
from getpass import getpass

user = "barnold"
pswd = quote(getpass('Database password: ')) # Prompt user for password; don't store in code. Use quote() to encode special characters in passwords
server = "at-database3.stanford.edu"
port = 3306
database = "enigma_acq"
tables = ["sessions", "stimulation", "behavior_traces"]


uri = f"mysql://{user}:{pswd}@{server}:{port}/{database}" 
dfs = {}
for table in tables:
    query = f"SELECT * FROM {table}"
    dfs[table] = pl.read_database_uri(query=query, uri=uri)

dfs['sessions']

## conditional assignment with window function

When multiple .when().then() statementes are used, Polars only considers a replacement expression that is deeper in the chain if the previous ones (predicates) all failed for that value.

In [ ]:
# accomplishing this in separate pieces

quantiles = (
   df.group_by(["species"]).agg(
        pl.col("swim_score").quantile(0.25).alias("q1_swim_score"),
        pl.col("swim_score").quantile(0.5).alias("q2_swim_score"),
        pl.col("swim_score").quantile(0.75).alias("q3_swim_score"),
    )
)
df_w_quant = df.join(quantiles, on="species")
df_w_quant = df_w_quant.with_columns(
    pl.when(pl.col("swim_score") <= pl.col("q1_swim_score"))
    .then(pl.lit("slow")) # need pl.lit() otherwise Polars treats it as a column
    .when(pl.col("swim_score") <= pl.col("q2_swim_score"))
    .then(pl.lit("intermediate"))
    .when(pl.col("swim_score") <= pl.col("q3_swim_score"))
    .then(pl.lit("fast"))
    .otherwise(pl.lit("really_fast"))
    .alias("swim_score_category")
).select(
    "species", "swim_score", "swim_score_category"
).sort(by="species")

df_w_quant.sample(10)


In [ ]:
# accomplishing this in a single step, using windows functions
df_categorized = df.with_columns(
    pl.col("swim_score").quantile(0.25).over("species").alias("q1"),
    pl.col("swim_score").quantile(0.5).over("species").alias("q2"), 
    pl.col("swim_score").quantile(0.75).over("species").alias("q3"),
).with_columns(
    pl.when(pl.col("swim_score") <= pl.col("q1"))
    .then(pl.lit("slow"))
    .when(pl.col("swim_score") <= pl.col("q2"))
    .then(pl.lit("intermediate"))
    .when(pl.col("swim_score") <= pl.col("q3"))
    .then(pl.lit("fast"))
    .otherwise(pl.lit("really_fast"))
    .alias("swim_score_category")
).drop(["q1", "q2", "q3"])  # Remove temporary quantile columns
df_categorized